In [1]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)


True

In [71]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("phanishankar134@gmail.com")  # Change to your verified sender
    to_email = To("pps130486@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [60]:
instructions1 = "You are a sales agent working for Microsoft, \
a company that provides a PaaS tool for building agents called Copilot Studio, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for Microsoft, \
a company that provides a PaaS tool for building agents called Copilot Studio, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for Microsoft, \
a company that provides a PaaS tool for building agents called Copilot Studio, powered by AI. \
You write concise, to the point cold emails."

In [61]:
sales_agent1 = Agent(name="ProfessionalSalesAgent",instructions=instructions1, model="gpt-4o-mini")
sales_agent2 = Agent(name="EngagingSalesAgent",instructions=instructions2, model="gpt-4o-mini")
sales_agent3 = Agent(name="ConciseSalesAgent",instructions=instructions3, model="gpt-4o-mini")

In [18]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Transform Your Development Experience with Microsoft Copilot Studio

Dear [Recipient's Name],

I hope this message finds you well.

I’m reaching out to introduce you to Microsoft Copilot Studio, our powerful Platform as a Service (PaaS) tool designed to streamline the development of intelligent agents. With the rapid evolution of AI, businesses must adapt quickly to remain competitive, and Copilot Studio can be a critical asset in that transition.

Our innovative platform empowers developers to create sophisticated AI-driven solutions with ease, enhancing productivity and reducing time to market. Key features include:

- **Rapid prototyping**: Build and iterate your agents quickly.
- **Customizable templates**: Leverage pre-built templates tailored for various industries.
- **Seamless integration**: Connect easily with existing systems and data sources.

I would love to schedule a brief call to discuss how Copilot Studio can address your unique challenges and help drive your i

In [ ]:
message = "Write a cold sales email"

with trace("Professional Sales Agent"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )
outputs = [result.final_output for result in results]

for output in outputs:
    print("\n---\n")
    print(output)

In [29]:
sales_picker = Agent(
    name="SalesEmailPicker",
    instructions="You are an expert sales email evaluator. Given three different sales emails, \
    choose the best one based on professionalism, engagement, and conciseness. \
    Do not give any explanation and respond with the email and which email it is only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales picker"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )
outputs = [result.final_output for result in results]

emails_combined = "Cold Sales emails:\n\n".join(outputs)

best = await Runner.run(
    sales_picker, input=emails_combined
)
print("\n---\n")
print("Best sales email selected:\n")   
print(best.final_output)

In [32]:
sales_agent1

Agent(name='ProfessionalSalesAgent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for Microsoft, a company that provides a PaaS tool for building agents called Copilot Studio, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Tools and Agents Intercations

In [70]:
@function_tool
def send_email(body: str):
    """Send  out an email with the given body to all sales prospects."""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("phanishankar134@gmail.com")  # Change to your verified sender
    to_email = To("pps130486@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status_code": response.status_code}

In [42]:
send_email

FunctionTool(name='send_email', description='Send  out an email with the given body to all sales prospects.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F9D89C6020>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

## Converting Agent into  a tool

In [43]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F9D89C5760>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### Gather all tools together

In [62]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F9D4982480>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F9D876C4A0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', descr

### Sales Manager - Planning agent

In [45]:
instructions = """You are a Sales Manager agent working for Microsoft, 
a company that provides a PaaS tool for building agents called Copilot Studio, powered by AI. 
Your job is to plan and execute a sales outreach campaign using the tools provided to you.     
You have access to the following tools: (tool1, tool2, tool3, send_email). You try all 3 tools once before choosing the best email to send to the prospect list.
You pick the best email among the 3 and use the send_email tool to send it out to all prospects. 
You must use the tools to complete your task."""

sales_manager = Agent(
    name="SalesManager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini"
)
message = "Your task is to write and send out a cold sales email to all prospects."

with trace("Sales Manager Agent"):
    result = await Runner.run(sales_manager, input=message)


##Hand offs between agents

###Handoffs represent a way an agent can delegate to an agent, passing control to it

In [63]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [64]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("phanishankar134@gmail.com")  # Change to your verified sender
    to_email = To("pps130486@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [65]:
tools_html = [subject_tool, html_tool, send_html_email]
tools_html

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F9D4982E80>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F9D876E840>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=No

In [66]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools_html,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [67]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F9D4982480>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F9D876C4A0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', descrip

In [68]:
sales_manager_instructions = """
You are a Sales Manager at Microsoft. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.

 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)